In [65]:
import torch
from config.paths import MODEL_PATHS
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [66]:
import os
import numpy as np
from scipy.io import loadmat
from config.paths import TAG_FILED_DATA_PATH

X_target_list = []

for file in os.listdir(TAG_FILED_DATA_PATH):
    if file.endswith(".mat"):
        mat_data = loadmat(os.path.join(TAG_FILED_DATA_PATH, file))
        print(file, mat_data.keys())

        # 获取除 __header__, __version__, __globals__ 外的键
        data_keys = [k for k in mat_data.keys() if not k.startswith('__')]
        if len(data_keys) != 1:
            raise ValueError(f"文件 {file} 中信号键不唯一: {data_keys}")
        signal_key = data_keys[0]

        # 读取信号
        signal = mat_data[signal_key]
        X_target_list.append(signal)

# 拼接成一个大数组
X_target_raw = np.vstack(X_target_list)
print("目标域数据 shape:", X_target_raw.shape)


A.mat dict_keys(['__header__', '__version__', '__globals__', 'A'])
B.mat dict_keys(['__header__', '__version__', '__globals__', 'B'])
C.mat dict_keys(['__header__', '__version__', '__globals__', 'C'])
D.mat dict_keys(['__header__', '__version__', '__globals__', 'D'])
E.mat dict_keys(['__header__', '__version__', '__globals__', 'E'])
F.mat dict_keys(['__header__', '__version__', '__globals__', 'F'])
G.mat dict_keys(['__header__', '__version__', '__globals__', 'G'])
H.mat dict_keys(['__header__', '__version__', '__globals__', 'H'])
I.mat dict_keys(['__header__', '__version__', '__globals__', 'I'])
J.mat dict_keys(['__header__', '__version__', '__globals__', 'J'])
K.mat dict_keys(['__header__', '__version__', '__globals__', 'K'])
L.mat dict_keys(['__header__', '__version__', '__globals__', 'L'])
M.mat dict_keys(['__header__', '__version__', '__globals__', 'M'])
N.mat dict_keys(['__header__', '__version__', '__globals__', 'N'])
O.mat dict_keys(['__header__', '__version__', '__globals__', '

In [67]:
from sklearn.model_selection import train_test_split

# 无标签
X_train_tgt, X_test_tgt = train_test_split(X_target_raw, test_size=0.2, random_state=42)
X_train_tgt, X_val_tgt = train_test_split(X_train_tgt, test_size=0.2, random_state=42)

print("训练集:", X_train_tgt.shape)
print("验证集:", X_val_tgt.shape)
print("测试集:", X_test_tgt.shape)


训练集: (2621440, 1)
验证集: (655360, 1)
测试集: (819200, 1)


In [68]:
import joblib
import os

TARGET_PICKLE_PATHS = r"E:/python_file/TransBearingDiag/src/data/target/processed"

save_file = os.path.join(TARGET_PICKLE_PATHS, "target_data.pkl")
# 保存目标域数据
joblib.dump([X_train_tgt, X_val_tgt, X_test_tgt], save_file)
print("目标域数据已保存:", save_file)


目标域数据已保存: E:/python_file/TransBearingDiag/src/data/target/processed\target_data.pkl


In [69]:
def split_signal(data, frame_len=128, step=64):
    frames = []
    for i in range(0, len(data) - frame_len + 1, step):
        frames.append(data[i:i+frame_len, 0])
    return np.array(frames)

frame_len = 128
step = 64
X_train_tgt_frames = split_signal(X_train_tgt, frame_len=frame_len, step=step)
X_val_tgt_frames   = split_signal(X_val_tgt, frame_len=frame_len, step=step)
X_test_tgt_frames  = split_signal(X_test_tgt, frame_len=frame_len, step=step)
print("分帧后目标域训练集:", X_train_tgt_frames.shape)

分帧后目标域训练集: (40959, 128)


In [70]:
from torch.utils.data import Dataset, DataLoader


class TargetDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

batch_size = 64
train_loader_tgt = DataLoader(TargetDataset(X_train_tgt_frames), batch_size=batch_size, shuffle=True)
val_loader_tgt   = DataLoader(TargetDataset(X_val_tgt_frames), batch_size=batch_size, shuffle=False)
test_loader_tgt  = DataLoader(TargetDataset(X_test_tgt_frames), batch_size=batch_size, shuffle=False)

print("目标域 DataLoader 已创建")

目标域 DataLoader 已创建


In [71]:
from sklearn.preprocessing import LabelEncoder

split_file = os.path.join(MODEL_PATHS, "train_val_test_split.pkl")
X_train_src, X_val_src, X_test_src, y_train_src, y_val_src, y_test_src = joblib.load(split_file)

# 标签编码
if isinstance(y_train_src, np.ndarray) and (y_train_src.dtype=='O' or y_train_src.dtype.type is np.str_):
    label_encoder = LabelEncoder()
    y_train_src = label_encoder.fit_transform(y_train_src)
    y_val_src   = label_encoder.transform(y_val_src)
    y_test_src  = label_encoder.transform(y_test_src)

y_train_src = torch.tensor(y_train_src, dtype=torch.long)
y_val_src   = torch.tensor(y_val_src, dtype=torch.long)
y_test_src  = torch.tensor(y_test_src, dtype=torch.long)

In [72]:
try:
    X_train_tgt, X_val_tgt, X_test_tgt = joblib.load(os.path.join(TARGET_PICKLE_PATHS, "target_data.pkl"))
    y_train_tgt = y_val_tgt = y_test_tgt = None
    print("目标域数据加载完成，无标签")
except:
    X_train_tgt = X_val_tgt = X_test_tgt = None
    print("未找到目标域数据")


目标域数据加载完成，无标签


In [73]:
# 处理源域标签
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch
if 'y_train_src' not in globals():
    split_file = os.path.join(MODEL_PATHS, "train_val_test_split.pkl")
    X_train_src, X_val_src, X_test_src, y_train_src, y_val_src, y_test_src = joblib.load(split_file)

# 如果 y_train_src 是 numpy array
if isinstance(y_train_src, np.ndarray):
    if y_train_src.dtype == 'O' or y_train_src.dtype.type is np.str_:
        label_encoder = LabelEncoder()
        y_train_src = label_encoder.fit_transform(y_train_src)
        y_val_src   = label_encoder.transform(y_val_src)
        y_test_src  = label_encoder.transform(y_test_src)
        print("源域类别映射：", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

elif isinstance(y_train_src, torch.Tensor):
    # 先转换为 numpy，再处理
    y_train_src_np = y_train_src.cpu().numpy()
    y_val_src_np   = y_val_src.cpu().numpy()
    y_test_src_np  = y_test_src.cpu().numpy()

    if y_train_src_np.dtype == 'O' or y_train_src_np.dtype.type is np.str_:
        label_encoder = LabelEncoder()
        y_train_src_np = label_encoder.fit_transform(y_train_src_np)
        y_val_src_np   = label_encoder.transform(y_val_src_np)
        y_test_src_np  = label_encoder.transform(y_test_src_np)
        print("源域类别映射：", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

    # 转回 torch.Tensor
    y_train_src = torch.tensor(y_train_src_np, dtype=torch.long)
    y_val_src   = torch.tensor(y_val_src_np, dtype=torch.long)
    y_test_src  = torch.tensor(y_test_src_np, dtype=torch.long)

# 转Tensor
else:
    y_train_src = torch.tensor(y_train_src, dtype=torch.long)
    y_val_src   = torch.tensor(y_val_src, dtype=torch.long)
    y_test_src  = torch.tensor(y_test_src, dtype=torch.long)
#源域数据 shape
print("源域训练集:", X_train_src.shape)
print("源域验证集:", X_val_src.shape)
print("源域测试集:", X_test_src.shape)

# 目标域数据 shape
if 'X_train_tgt' in globals() and X_train_tgt is not None:
    print("目标域训练集:", X_train_tgt.shape)
    print("目标域验证集:", X_val_tgt.shape)
    print("目标域测试集:", X_test_tgt.shape)
else:
    print("目标域数据不存在，DataLoader 未创建")


源域训练集: (96, 19)
源域验证集: (32, 19)
源域测试集: (33, 19)
目标域训练集: (2621440, 1)
目标域验证集: (655360, 1)
目标域测试集: (819200, 1)


In [74]:
import torch.nn as nn

class CNN1D(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(CNN1D, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv1d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        # 动态计算卷积输出
        with torch.no_grad():
            x = torch.zeros(1, 1, input_dim)
            x = self.conv_layers(x)
            self._to_linear = x.numel()

        self.fc_layers = nn.Sequential(
            nn.Linear(self._to_linear, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

input_dim = X_train_tgt_frames.shape[1]
num_classes = len(torch.unique(y_train_src))
model_tgt = CNN1D(input_dim, num_classes)


In [75]:
model_tgt = CNN1D(input_dim, num_classes)
src_model_path = os.path.join(MODEL_PATHS, "model_src.pth")
pretrained_dict = torch.load(src_model_path, map_location='cpu')
# 保留卷积层权重
pretrained_dict = {k: v for k, v in pretrained_dict.items() if "conv_layers" in k}
model_dict = model_tgt.state_dict()
model_dict.update(pretrained_dict)
model_tgt.load_state_dict(model_dict)

<All keys matched successfully>

In [76]:
src_model_path = os.path.join(MODEL_PATHS, "model_src.pth")
pretrained_dict = torch.load(src_model_path, map_location='cpu')
model_dict = model_tgt.state_dict()
pretrained_dict = {k: v for k, v in pretrained_dict.items() if "conv_layers" in k and k in model_dict}
model_dict.update(pretrained_dict)
model_tgt.load_state_dict(model_dict)

for param in model_tgt.conv_layers.parameters():
    param.requires_grad = False

model_tgt.to(device)

CNN1D(
  (conv_layers): Sequential(
    (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
    (4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (6): ReLU()
    (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=2048, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=4, bias=True)
  )
)

In [77]:
# for param in model_tgt.conv_layers.parameters():
#     param.requires_grad = False
#
# model_tgt.to(device)

In [78]:
from tqdm import tqdm
from torch import optim


def train_target_model(model, train_loader, num_epochs=5, lr=1e-3):
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0
        for X in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            X = X.to(device)
            optimizer.zero_grad()
            outputs = model(X)  # [batch, num_classes]
            loss = torch.mean(outputs ** 2)  # L2 自监督
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * X.size(0)
        print(f"Epoch {epoch + 1}, Avg Loss: {running_loss / len(train_loader.dataset):.6f}")
    return model

In [79]:
num_epochs = 5
model_tgt = train_target_model(model_tgt, train_loader_tgt, num_epochs=num_epochs, lr=1e-3)

# 保存模型
torch.save(model_tgt.state_dict(), os.path.join(MODEL_PATHS, "model_tgt_notags.pth"))
torch.save(model_tgt, os.path.join(MODEL_PATHS, "full_model_tgt_notags.pth"))
print("目标域模型已保存")

Epoch 1/5: 100%|██████████| 640/640 [00:00<00:00, 824.47it/s]


Epoch 1, Avg Loss: 1.745627


Epoch 2/5: 100%|██████████| 640/640 [00:00<00:00, 930.10it/s] 


Epoch 2, Avg Loss: 0.000561


Epoch 3/5: 100%|██████████| 640/640 [00:00<00:00, 904.01it/s]


Epoch 3, Avg Loss: 0.000028


Epoch 4/5: 100%|██████████| 640/640 [00:00<00:00, 892.62it/s]


Epoch 4, Avg Loss: 0.000003


Epoch 5/5: 100%|██████████| 640/640 [00:00<00:00, 865.13it/s]

Epoch 5, Avg Loss: 0.000000
目标域模型已保存


In [80]:
model_tgt.eval()
for X in test_loader_tgt:
    X = X.to(device)
    with torch.no_grad():
        y_pred = model_tgt(X)
    print(y_pred.shape)


torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
torch.Size([64, 4])
